<a href="https://colab.research.google.com/github/wenlarry/Capstone-1/blob/master/Python_Everyone_Pt_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Spidering Twitter using a db

# Create a simple spidering program to go thru Twitter accounts n build a db. 

# For Spidering program - need to store data as we retrieve it

# Start by retrieving one person's Twitter friends n their statuses, looping thru the list of friends n adding
#  each of the friends to a db. After processing one person's Twitter friends, check db n retrieve one of the 
#  friends of the friend.
# Track how many times, a particular friend is in the db to get a sense of 'popularity'

# Db is stored in the file 'spider.sqlite3' that has one table 'Twitter'. Each row in the table has a col for
#  the account name, whether retrieved the friends of this account, and how many times the account has been
# 'friended'

# In the main loop , prompt the user for a Twitter account name or quit. If the user enters a Twitter account,
#  retrieve the list of friends n statuses for that user and add each friend to the db. If the friend is already
#  in the list, add 1 to the friends field in the row in the db
# If user press enter, look in the db for the next Twitter account that have not retrieved, retrieve the friends n
#  n statuses for that account, add to the db or update them n increase their friends count
# Once we retrieve the list of friends n statuses, loop thru all the user items in the returned JSON n retrieve
#  the screen_name for each user. Then use SELECT statement to see if we hv stored this particular screen_name
#  in the db n retrieve the friend count(friends) if the record exists.

# Once the sursor executes SELECT. retrieve the rows. Do this with a for statement. If retrieving on row(LIMIT 1),
#  use fetchone(). fetchone() returns the row as a tuple.Take the first value from the tuple using to get the
#  current friend count into the variable count.
# If retrieval is successful, use the SQL UPDATE with a WHERE clause to add 1 to the friends column for the row
#  that matches the freind's account. (Note: 2 place holders i.e. ??) in the SQL n the 2nd para to the execute()
#  is a 2 element tuple that hold the values to be substituted into the SQL in place of the ??.
# If the code in the try block fails, it is probably because no record matched the WHERE name = ? clause on the
#  SELECT. so in the except block, use the SQL INSERT to add friend's screen_name to the table with an indication
#  that we hv no retrieved the screen_name n set the friend count to zero

from urllib.request import urlopen
import urllib.error
#import twurl
import json
import sqlite3
import ssl

TWITTER_URL = 'https://api.twitter.com/1.1/friends/list.json'

conn = sqlite3.connect('spider.sqlite')
cur = conn.cursor()

cur.execute('''
            CREATE TABLE IF NOT EXISTS Twitter
            (name TEXT, retrieved INTEGER, friends INTEGER)''')

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

while True:
    acct = input('Enter a Twitter account, or quit: ')
    if (acct == 'quit'): break
    if (len(acct) < 1):
        cur.execute('SELECT name FROM Twitter WHERE retrieved = 0 LIMIT 1')
        try:
            acct = cur.fetchone()[0]
        except:
            print('No unretrieved Twitter accounts found')
            continue

    url = twurl.augment(TWITTER_URL, {'screen_name': acct, 'count': '20'})
    print('Retrieving', url)
    connection = urlopen(url, context=ctx)
    data = connection.read().decode()
    headers = dict(connection.getheaders())

    print('Remaining', headers['x-rate-limit-remaining'])
    js = json.loads(data)
    # Debugging
    # print json.dumps(js, indent=4)

    cur.execute('UPDATE Twitter SET retrieved=1 WHERE name = ?', (acct, ))

    countnew = 0
    countold = 0
    for u in js['users']:
        friend = u['screen_name']
        print(friend)
        cur.execute('SELECT friends FROM Twitter WHERE name = ? LIMIT 1',
                    (friend, ))
        try:
            count = cur.fetchone()[0]
            cur.execute('UPDATE Twitter SET friends = ? WHERE name = ?',
                        (count+1, friend))
            countold = countold + 1
        except:
            cur.execute('''INSERT INTO Twitter (name, retrieved, friends)
                        VALUES (?, 0, 1)''', (friend, ))
            countnew = countnew + 1
    print('New accounts=', countnew, ' revisited=', countold)
    conn.commit()

cur.close()


Enter a Twitter account, or quit: quit


In [2]:
# For the first time we run the program, the db is empty and a db in the file spider.sqlite 3 is created. A table
#  'Twitter' is added to the db. Some friends are retrieved and added to the empty db.
# Proceed to weite a simple db dumper to look at what is inside 'spided.sqlite2 file'
# This program opens the db n selects all the columns of all the rows in the table 'Twitter' then loops thru the
#  rows n prints each row.

import sqlite3

conn = sqlite3.connect('spider.sqlite')
cur = conn.cursor()
cur.execute('SELECT * FROM Twitter')
count = 0
for row in cur:
    print(row)
    count = count + 1
print(count, 'rows.')
cur.close()

# Output will be as follows
#('opencontent', 0,1)
#(heheiingold', 0,1)

# we get one row for each screen_name, that we hv not retrieved the data for that screen_name (shown as zero) n
#  everyone has one friend.
# Next our db reflects the retrieval of the friends of our first Twitter account. Run the program againn get it
#  to retrieve the friends of the next unprocessed account by 'enter' instead of a Twitter account


0 rows.


In [3]:
# Cont'd from above. As we press enter, the following code is executed
# Use the SQL Select to retrieve the name of the first (LIMIT 1) user who still has their 'have we retrieved this
#  user' value set to zero. Also use fetchone()[0] pattern within try/except block to extract a screen_name from
#  the retrieved data or trigger an error message n loop back up.

#if (len(acct) < 1):
        #cur.execute('SELECT name FROM Twitter WHERE retrieved = 0 LIMIT 1')
        #try:
            #acct = cur.fetchone()[0]
        #except:
            #print('No unretrieved Twitter accounts found')
            #continue
            
# If we successfully retrieved an unprocessed screen-name, we retrieve their data as follows:

 #url = twurl.augment(TWITTER_URL, {'screen_name': acct, 'count': '20'})
    #print('Retrieving', url)
    #connection = urllib.urlopen(url)
    #data = connection.read()
    # js = json.loads(data)
    
    #cur.execute('UPDATE Twitter SET retrieved=1 WHERE name = ?', (acct, ))
    
# Once the data is successully retrieved, use the UPDATE statement to set the retrieved column to 1 to indicate
#  that we hv completed the retrieval of the friends in this account. this prevents retrieving the data over n over.
# Run the friend program n press enter twice to retrieve the next unvisited friend's friends then run the dumping
#  program

# Output:

  #('opencontent', 1,1)
  #('kthanos', 0,2)
  #('LectureTools', 0,1)

# Properly recorded that we hv visited 'opencontent'. Also, 'kthaos' already hv 2 followers
# Each time we run the program n press enter it will pick the next unvisited account, retireve their friends,
#  marked them as retrieved n for each of the friends of the next account wither add them to the database or
#  update their friend account if they are already in the db.


In [4]:
# Basic Data Modeling

# Power of a relatiional db lies in the creation of multiple tables n link these tables. How to break application
#  data into multiple tables n establishing the relationship btw the tables is 'data modeling'

# For the spider Twitter spider application, instead of just counting a person's friends, we want to keep a list
#  of all the incoming relationships so we could find a list of everyone who is following a particulat account
#  We create a new table that tracks pairs of friends. Code below:

#CREATE TABLE Pals(from_friend TEXT, to_friend TEXT)

# Each time we encounter a person who Ming is followig, we would insert a row.

#INSERT INTO Pals(from_friend,to_friend) VALUES ('ming', 'anon')

# A string takes up more space than an integer in the computer's memory n takes more processor time to compare n
#  sort. If we hv lots of data in the db n links, it is important to scan data quickly. Therefore, store Twitter
#  accounts in a table ' People' instead of the previous table 'Twitter'. The 'People' table has an additional
#  col to store the numeric key assoicated with the row for the Twitter user. SQLite has a feature that 
#  automatically adds the key value for any row inserted into a table using a special type of data col
#  (INTEGER PRIMARY KEY)

#CREATE TABLE People
    #(id INTEGER PRIMARY KEY, name TEXT UNIQUE, retrieved INTEGER)
    
# No longer maintaining a friend count in each row of the PEOPLE table. INTEGER PRIMARY KEY - type of id col,
#  indicating that SQLite to manage this clo n assign a unique numeric key to each row inserted automatically.
#  Add the keyword UNIQUE to indicate that we not allow SQLite to insert 2 row with the same value for name

# Instead of creating the table 'Pals', create a table 'Follows' with 2 integer col 'from_id' and 'to_id' n
#  a constraint on the table that the combination of 'from_id' and 'to_id' must be unique in this table.

#CREATE TABLE Follows
    #(from_id INTEGER, to_ID INTEGER, UNIQUE(from_id, to_id))
    
# When UNIQUE clause is added to the table, we r communicating a set of rules that we are getting the db to
#  enforce when we insert records. The rules are to avoid mistakes n to simplify coding.
# In creating the 'Follows' table, we are modeliing a relationship where one person 'follows' someone else n
#  representing it with a pair of numbers indicating that a) the people are connected and b) the direction of 
#  the realtionship.


In [5]:
# Programming with multiple tables

# Twitter spider program using 2 tables, the primary keys n the key references

# Create tables with primary keys n constraints
# When we hv a logical key for a person (i.e. account name) n we need the id value for the person, depending
#  on whether or not the person is already in the People table, we need to 1) look up the person is already in the
#  People table n retrieve the id value for the person or 2)add the person to the People table n get the id value
#  for the newly added row
# Insert the row that captures the 'follows' relationship
# Programming with multiple tables

import urllib.request, urllib.parse, urllib.error
#import twurl
import json
import sqlite3
import ssl

TWITTER_URL = 'https://api.twitter.com/1.1/friends/list.json'

conn = sqlite3.connect('friends.sqlite')
cur = conn.cursor()

cur.execute('''CREATE TABLE IF NOT EXISTS People
            (id INTEGER PRIMARY KEY, name TEXT UNIQUE, retrieved INTEGER)''')
cur.execute('''CREATE TABLE IF NOT EXISTS Follows
            (from_id INTEGER, to_id INTEGER, UNIQUE(from_id, to_id))''')

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

while True:
    acct = input('Enter a Twitter account, or quit: ')
    if (acct == 'quit'): break
    if (len(acct) < 1):
        cur.execute('SELECT id, name FROM People WHERE retrieved=0 LIMIT 1')
        try:
            (id, acct) = cur.fetchone()
        except:
            print('No unretrieved Twitter accounts found')
            continue
    else:
        cur.execute('SELECT id FROM People WHERE name = ? LIMIT 1',
                    (acct, ))
        try:
            id = cur.fetchone()[0]
        except:
            cur.execute('''INSERT OR IGNORE INTO People
                        (name, retrieved) VALUES (?, 0)''', (acct, ))
            conn.commit()
            if cur.rowcount != 1:
                print('Error inserting account:', acct)
                continue
            id = cur.lastrowid

    url = twurl.augment(TWITTER_URL, {'screen_name': acct, 'count': '100'})
    print('Retrieving account', acct)
    try:
        connection = urllib.request.urlopen(url, context=ctx)
    except Exception as err:
        print('Failed to Retrieve', err)
        break

    data = connection.read().decode()
    headers = dict(connection.getheaders())

    print('Remaining', headers['x-rate-limit-remaining'])

    try:
        js = json.loads(data)
    except:
        print('Unable to parse json')
        print(data)
        break

    # Debugging
    # print(json.dumps(js, indent=4))

    if 'users' not in js:
        print('Incorrect JSON received')
        print(json.dumps(js, indent=4))
        continue

    cur.execute('UPDATE People SET retrieved=1 WHERE name = ?', (acct, ))

    countnew = 0
    countold = 0
    for u in js['users']:
        friend = u['screen_name']
        print(friend)
        cur.execute('SELECT id FROM People WHERE name = ? LIMIT 1',
                    (friend, ))
        try:
            friend_id = cur.fetchone()[0]
            countold = countold + 1
        except:
            cur.execute('''INSERT OR IGNORE INTO People (name, retrieved)
                        VALUES (?, 0)''', (friend, ))
            conn.commit()
            if cur.rowcount != 1:
                print('Error inserting account:', friend)
                continue
            friend_id = cur.lastrowid
            countnew = countnew + 1
        cur.execute('''INSERT OR IGNORE INTO Follows (from_id, to_id)
                    VALUES (?, ?)''', (id, friend_id))
    print('New accounts=', countnew, ' revisited=', countold)
    print('Remaining', headers['x-rate-limit-remaining'])
    conn.commit()
cur.close()


Enter a Twitter account, or quit: quit


In [6]:
# Constraints in db tables

# When designing table structures, tell db to enforce few rules. Rules help with errors n including incorrect data
#  into tables
# Indicate that the name col in the People table must be UNIQUE. Indicate that combination of 2 numbers in each row
#  of the Follows table be unique. Help with avoiding errors such as adding the same relationship twice.

#cur.execute('''CREATE TABLE IF NOT EXISTS People
            #(id INTEGER PRIMARY KEY, name TEXT UNIQUE, retrieved INTEGER)''')
#cur.execute('''CREATE TABLE IF NOT EXISTS Follows
            #(from_id INTEGER, to_id INTEGER, UNIQUE(from_id, to_id))''')

# Add 'OR IGNORE' clause to INSERT statement to indicate that if this particular INSERT would cause a violation 
#  of the 'name must be unique' rule, the db system would ignore the INSERT.

#cur.execute('''INSERT OR IGNORE INTO People (name, retrieved) 
   # VALUES (?, 0)''', (friend, ))

# Code ensures that we don't add the exact same Follows relationship twice. Again, tell db to ignore INSERT if it
#  violates the uniquess constraint specified for the Follows rows

#cur.execute('''INSERT OR IGNORE INTO Follows 
   # (from_id, to_id) VALUES (?, ?)''', (id, friend_id))

In [7]:
# Retrieve and/or insert a record

# Prompt the user for a Twitter account, if the account exists, look up its id value. If account does not yet exist
#  in the People table, insert the record n get the id value from the inserted row
# Code shows the look up of the id for a friend's account when we hv extracted a screen_name from a user node in the
#  retrieved Twitter JSON. 
# Check if the People record exists using a SELECT statement. If SELECT fails, the fetchone()[0] code will fail n
#  control will transfer to the except section
# If end up in except code, it means that the row is not found, so insert the row. Use 'INSERT OR IGNORE' just to
#  avoid errors n then commit(0) to get db to be updated. 
# 'cur.rowcount' - check how many rows affected. Attempt is to insert a single row, if the number of affected rows
#  is more than 1, it is an error
# If INSERT is successful, goto 'cur.lastrowid' to find the value the db assigned to the id col in the new row.

#friend = u['screen_name']
#cur.execute('SELECT id FROM People WHERE name = ? LIMIT 1',
    #(friend, ))
#try:
    #friend_id = cur.fetchone()[0]
    #countold = countold + 1
#except:
    #cur.execute('''INSERT OR IGNORE INTO People (name, retrieved)
        #VALUES (?, 0)''', (friend, ))
    #conn.commit()
    #if cur.rowcount != 1:
        #print('Error inserting account:', friend)
        #continue
    #friend_id = cur.lastrowid
    #countnew = countnew + 1


In [8]:
# Storing the friend relationship

# Once the key value for both the Twitter user n the friend in the JSON  are available, just insert the 2 numbers
#  into the Follows table

# cur.execute('''INSERT OR IGNORE INTO Follows (from_id, to_id) VALUES (?, ?)''', (id, friend_id))

# Output: id, name n visited fields in the People table; numbers of both ends in the relationship in the Follows
#  table. In the People table, the first 2 people hv been visited. Data in the Follows table indicates that 'drchuck'
#  (user 1) is friend to all of the people in the first 2 rows. 

# People:
# (1, 'drchuck', 1)
# (2, ming', 0)
# Follows:
# (1,2)
# (1,2)


In [9]:
# Three kinds of keys in a db model

# logical key - key that the real world use to look up a row, name field is a logical key. It is the screen name
#  for the user. Make sense to add a UNIQUE constraint to a logical key.

# Primary key - Usually a number automatically assigned by the db. Seraching for the row using the primary key is
#  fastest. id field is an excample

# foreign key - Usually a number that points to the primary key of an associate row in a different table. eg from_id


In [10]:
# Using JOIN to retrieve data

# With data separated into 2 tables, linked together using primary and foreign keys, build a SELECT that resembles
#  the data across the tables.
# SQL uses the JOIN clause to reconnect these tables. In the JOINT clause, specify the fields that are used to 
#  reconnect the rows btw the tables

# JOIN clause indicates that the fields selected cross both the Follows and People tables. ON clause indicates how the
#  2 tables are to be joined. Take the rows from Follows and append the row from People where the field
#  from_id in Follows is the same id value in the People table.
# Result of JOIN is to create extra-long 'metarows' which hv both the fields from People and the matching fields from
#  Follows. Where there is more than one match btw the id field from People and the from_id from People then JOIN
#  creates a metarow for rach of the matching pairs of rows, duplicating data as needed

# EG:
# SELECT * FROM Follows JOIN People
#     ON Follows.from_id = People.id WHERE People.id =1


In [11]:
# Following code shows the data in the db after the multi-table Twitter program (above) has been run several times.
# In this program - 1)dump out the People and Follows n then dump out a subset in the tables joined together.

# Actual Output:
# python twjoin.py
# People:
#(1 'drchuck', 1)
#(4 'steve_coppin', 0)
# Follows:
#(1,2)
#(1,5)
#Connections for id_2:
#(2,1,1,'drchuck',1)
#(2,102,102,'SomethinGirl', 0)

import sqlite3

conn = sqlite3.connect('friends.sqlite')
cur = conn.cursor()

cur.execute('SELECT * FROM People')
count = 0
print('People:')
for row in cur:
    if count < 5: print(row)
    count = count + 1
print(count, 'rows.')

cur.execute('SELECT * FROM Follows')
count = 0
print('Follows:')
for row in cur:
    if count < 5: print(row)
    count = count + 1
print(count, 'rows.')

cur.execute('''SELECT * FROM Follows JOIN People
            ON Follows.to_id = People.id
            WHERE Follows.from_id = 2''')
count = 0
print('Connections for id=2:')
for row in cur:
    if count < 5: print(row)
    count = count + 1
print(count, 'rows.')

cur.close()

People:
0 rows.
Follows:
0 rows.
Connections for id=2:
0 rows.


In [12]:
# Summary

# More complicate to write the code to use db to store data than Python dict.
# Db useful - 1) when application needs to make small many random updates within a large data set, 2)when data is
#             large n cannot fit in a dict n need to look up info repeatedly, or 3)a long-running process that can
#             be stopped n restarted n retained the data from one run to another.

# Can build a simple db with a single table to suit many applications but most problems require several tables n
#  links/relationships btw rows in different tables. When making links btw tables, do thoughtful design n follow the 
#  rules of db normalization. Model data efficiently so the program run as fast as possible

In [13]:
# Debugging

# When developing a Python program to connect to an SQLite db will be to run a Python program n check the results
#  using the DB Browser fro SQLite. Browser allows a check to see if the program is working
# Careful - SQLite keep 2 programs from changing that same data at the same time. Eg. if you open a db in the
#  browser n make a change to the db n hv not yet 'save' in the browser, the browser 'locks' the db file n keeps
#  any other program from accessing the file. Python cannot access the file if it is locked. Solution is to either
#  close the db in the browser b4 attempting to access the db from Python

In [14]:
# Visualizing mail data

# Download: www.py43.com/code3/gmane.zip

Analyzing an EMAIL Archive from gmane and vizualizing the data
using the D3 JavaScript library

This is a set of tools that allow you to pull down an archive
of a gmane repository using the instructions at:

http://gmane.org/export.php

In order not to overwhelm the gmane.org server, I have put up 
my own copy of the messages at: 

http://mbox.dr-chuck.net/

This server will be faster and take a lot of load off the 
gmane.org server.

You should install the SQLite browser to view and modify the databases from:

http://sqlitebrowser.org/

The first step is to spider the gmane repository.  The base URL 
is hard-coded in the gmane.py and is hard-coded to the Sakai
developer list.  You can spider another repository by changing that
base url.   Make sure to delete the content.sqlite file if you 
switch the base url.  The gmane.py file operates as a spider in 
that it runs slowly and retrieves one mail message per second so 
as to avoid getting throttled by gmane.org.   It stores all of
its data in a database and can be interrupted and re-started 
as often as needed.   It may take many hours to pull all the data
down.  So you may need to restart several times.

To give you a head-start, I have put up 600MB of pre-spidered Sakai 
email here:

https://www.py4e.com/data_space/content.sqlite.zip

If you download this, you can "catch up with the latest" by
running gmane.py.

Navigate to the folder where you extracted the gmane.zip

Note: Windows has difficulty in displaying UTF-8 characters
in the console so for each console window you open, you may need
to type the following command before running this code:

    chcp 65001

http://stackoverflow.com/questions/388490/unicode-characters-in-windows-command-line-how

Here is a run of gmane.py getting the last five messages of the
sakai developer list:

Mac: python3 gmane.py 
Win: gmane.py 

How many messages:10
http://mbox.dr-chuck.net/sakai.devel/1/2 2662
    ggolden@umich.edu 2005-12-08T23:34:30-06:00 call for participation: developers documentation
http://mbox.dr-chuck.net/sakai.devel/2/3 2434
    csev@umich.edu 2005-12-09T00:58:01-05:00 report from the austin conference:  sakai developers break into song
http://mbox.dr-chuck.net/sakai.devel/3/4 3055
    kevin.carpenter@rsmart.com 2005-12-09T09:01:49-07:00 cas and sakai 1.5
http://mbox.dr-chuck.net/sakai.devel/4/5 11721
    michael.feldstein@suny.edu 2005-12-09T09:43:12-05:00 re: lms/vle rants/comments
http://mbox.dr-chuck.net/sakai.devel/5/6 9443
    john@caret.cam.ac.uk 2005-12-09T13:32:29+00:00 re: lms/vle rants/comments
Does not start with From 

The program scans content.sqlite from 1 up to the first message number not
already spidered and starts spidering at that message.  It continues spidering
until it has spidered the desired number of messages or it reaches a page
that does not appear to be a properly formatted message.

Sometimes gmane.org is missing a message.  Perhaps administrators can delete messages
or perhaps they get lost - I don't know.   If your spider stops, and it seems it has hit
a missing message, go into the SQLite Manager and add a row with the missing id - leave
all the other fields blank - and then restart gmane.py.   This will unstick the 
spidering process and allow it to continue.  These empty messages will be ignored in the next
phase of the process.

One nice thing is that once you have spidered all of the messages and have them in 
content.sqlite, you can run gmane.py again to get new messages as they get sent to the
list.  gmane.py will quickly scan to the end of the already-spidered pages and check 
if there are new messages and then quickly retrieve those messages and add them 
to content.sqlite.

The content.sqlite data is pretty raw, with an innefficient data model, and not compressed.
This is intentional as it allows you to look at content.sqlite to debug the process.
It would be a bad idea to run any queries against this database as they would be 
slow.

The second process is running the program gmodel.py.  gmodel.py reads the rough/raw 
data from content.sqlite and produces a cleaned-up and well-modeled version of the 
data in the file index.sqlite.  The file index.sqlite will be much smaller (often 10X
smaller) than content.sqlite because it also compresses the header and body text.

Each time gmodel.py runs - it completely wipes out and re-builds index.sqlite, allowing
you to adjust its parameters and edit the mapping tables in content.sqlite to tweak the 
data cleaning process.

Running gmodel.py works as follows:

Mac: python3 gmodel.py
Win: gmodel.py

Loaded allsenders 1588 and mapping 28 dns mapping 1
1 2005-12-08T23:34:30-06:00 ggolden22@mac.com
251 2005-12-22T10:03:20-08:00 tpamsler@ucdavis.edu
501 2006-01-12T11:17:34-05:00 lance@indiana.edu
751 2006-01-24T11:13:28-08:00 vrajgopalan@ucmerced.edu
...

The gmodel.py program does a number of data cleaing steps

Domain names are truncated to two levels for .com, .org, .edu, and .net 
other domain names are truncated to three levels.  So si.umich.edu becomes
umich.edu and caret.cam.ac.uk becomes cam.ac.uk.   Also mail addresses are
forced to lower case and some of the @gmane.org address like the following

   arwhyte-63aXycvo3TyHXe+LvDLADg@public.gmane.org

are converted to the real address whenever there is a matching real email
address elsewhere in the message corpus.

If you look in the content.sqlite database there are two tables that allow
you to map both domain names and individual email addresses that change over 
the lifetime of the email list.  For example, Steve Githens used the following
email addresses over the life of the Sakai developer list:

s-githens@northwestern.edu
sgithens@cam.ac.uk
swgithen@mtu.edu

We can add two entries to the Mapping table

s-githens@northwestern.edu ->  swgithen@mtu.edu
sgithens@cam.ac.uk -> swgithen@mtu.edu

And so all the mail messages will be collected under one sender even if 
they used several email addresses over the lifetime of the mailing list.

You can also make similar entries in the DNSMapping table if there are multiple
DNS names you want mapped to a single DNS.  In the Sakai data I add the following
mapping:

iupui.edu -> indiana.edu

So all the folks from the various Indiana University campuses are tracked together

You can re-run the gmodel.py over and over as you look at the data, and add mappings
to make the data cleaner and cleaner.   When you are done, you will have a nicely
indexed version of the email in index.sqlite.   This is the file to use to do data
analysis.   With this file, data analysis will be really quick.

The first, simplest data analysis is to do a "who does the most" and "which 
organzation does the most"?  This is done using gbasic.py:

Mac: python3 gbasic.py 
Win: gbasic.py 

How many to dump? 5
Loaded messages= 51330 subjects= 25033 senders= 1584

Top 5 Email list participants
steve.swinsburg@gmail.com 2657
azeckoski@unicon.net 1742
ieb@tfd.co.uk 1591
csev@umich.edu 1304
david.horwitz@uct.ac.za 1184

Top 5 Email list organizations
gmail.com 7339
umich.edu 6243
uct.ac.za 2451
indiana.edu 2258
unicon.net 2055

You can look at the data in index.sqlite and if you find a problem, you 
can update the Mapping table and DNSMapping table in content.sqlite and
re-run gmodel.py.

There is a simple vizualization of the word frequence in the subject lines
in the file gword.py:

Mac: python3 gword.py
Win: gword.py

Range of counts: 33229 129
Output written to gword.js

This produces the file gword.js which you can visualize using the file 
gword.htm.

A second visualization is in gline.py.  It visualizes email participation by 
organizations over time.

Mac: python3 gline.py 
Win: gline.py 

Loaded messages= 51330 subjects= 25033 senders= 1584
Top 10 Oranizations
['gmail.com', 'umich.edu', 'uct.ac.za', 'indiana.edu', 'unicon.net', 'tfd.co.uk', 'berkeley.edu', 'longsight.com', 'stanford.edu', 'ox.ac.uk']
Output written to gline.js

Its output is written to gline.js which is visualized using gline.htm.

Some URLs for visualization ideas:

https://developers.google.com/chart/

https://developers.google.com/chart/interactive/docs/gallery/motionchart

https://code.google.com/apis/ajax/playground/?type=visualization#motion_chart_time_formats

https://developers.google.com/chart/interactive/docs/gallery/annotatedtimeline

http://bost.ocks.org/mike/uberdata/

http://mbostock.github.io/d3/talk/20111018/calendar.html

http://nltk.org/install.html

As always - comments welcome.

-- Dr. Chuck

SyntaxError: ignored

In [18]:
# gmane.py

import sqlite3
import time
import ssl
import urllib.request, urllib.parse, urllib.error
from urllib.parse import urljoin
from urllib.parse import urlparse
import re
from datetime import datetime, timedelta

# Not all systems have this so conditionally define parser
try:
    import dateutil.parser as parser
except:
    pass

def parsemaildate(md) :
    # See if we have dateutil
    try:
        pdate = parser.parse(tdate)
        test_at = pdate.isoformat()
        return test_at
    except:
        pass

    # Non-dateutil version - we try our best

    pieces = md.split()
    notz = " ".join(pieces[:4]).strip()

    # Try a bunch of format variations - strptime() is *lame*
    dnotz = None
    for form in [ '%d %b %Y %H:%M:%S', '%d %b %Y %H:%M:%S',
        '%d %b %Y %H:%M', '%d %b %Y %H:%M', '%d %b %y %H:%M:%S',
        '%d %b %y %H:%M:%S', '%d %b %y %H:%M', '%d %b %y %H:%M' ] :
        try:
            dnotz = datetime.strptime(notz, form)
            break
        except:
            continue

    if dnotz is None :
        # print 'Bad Date:',md
        return None

    iso = dnotz.isoformat()

    tz = "+0000"
    try:
        tz = pieces[4]
        ival = int(tz) # Only want numeric timezone values
        if tz == '-0000' : tz = '+0000'
        tzh = tz[:3]
        tzm = tz[3:]
        tz = tzh+":"+tzm
    except:
        pass

    return iso+tz

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

conn = sqlite3.connect('content.sqlite')
cur = conn.cursor()

baseurl = "http://mbox.dr-chuck.net/sakai.devel/"

cur.execute('''CREATE TABLE IF NOT EXISTS Messages
    (id INTEGER UNIQUE, email TEXT, sent_at TEXT,
     subject TEXT, headers TEXT, body TEXT)''')

# Pick up where we left off
start = None
cur.execute('SELECT max(id) FROM Messages' )
try:
    row = cur.fetchone()
    if row is None :
        start = 0
    else:
        start = row[0]
except:
    start = 0

if start is None : start = 0

many = 0
count = 0
fail = 0
while True:
    if ( many < 1 ) :
        conn.commit()
        sval = input('How many messages:')
        if ( len(sval) < 1 ) : break
        many = int(sval)

    start = start + 1
    cur.execute('SELECT id FROM Messages WHERE id=?', (start,) )
    try:
        row = cur.fetchone()
        if row is not None : continue
    except:
        row = None

    many = many - 1
    url = baseurl + str(start) + '/' + str(start + 1)

    text = "None"
    try:
        # Open with a timeout of 30 seconds
        document = urllib.request.urlopen(url, None, 30, context=ctx)
        text = document.read().decode()
        if document.getcode() != 200 :
            print("Error code=",document.getcode(), url)
            break
    except KeyboardInterrupt:
        print('')
        print('Program interrupted by user...')
        break
    except Exception as e:
        print("Unable to retrieve or parse page",url)
        print("Error",e)
        fail = fail + 1
        if fail > 5 : break
        continue

    print(url,len(text))
    count = count + 1

    if not text.startswith("From "):
        print(text)
        print("Did not find From ")
        fail = fail + 1
        if fail > 5 : break
        continue

    pos = text.find("\n\n")
    if pos > 0 :
        hdr = text[:pos]
        body = text[pos+2:]
    else:
        print(text)
        print("Could not find break between headers and body")
        fail = fail + 1
        if fail > 5 : break
        continue

    email = None
    x = re.findall('\nFrom: .* <(\S+@\S+)>\n', hdr)
    if len(x) == 1 :
        email = x[0];
        email = email.strip().lower()
        email = email.replace("<","")
    else:
        x = re.findall('\nFrom: (\S+@\S+)\n', hdr)
        if len(x) == 1 :
            email = x[0];
            email = email.strip().lower()
            email = email.replace("<","")

    date = None
    y = re.findall('\Date: .*, (.*)\n', hdr)
    if len(y) == 1 :
        tdate = y[0]
        tdate = tdate[:26]
        try:
            sent_at = parsemaildate(tdate)
        except:
            print(text)
            print("Parse fail",tdate)
            fail = fail + 1
            if fail > 5 : break
            continue

    subject = None
    z = re.findall('\Subject: (.*)\n', hdr)
    if len(z) == 1 : subject = z[0].strip().lower();

    # Reset the fail counter
    fail = 0
    print("   ",email,sent_at,subject)
    cur.execute('''INSERT OR IGNORE INTO Messages (id, email, sent_at, subject, headers, body)
        VALUES ( ?, ?, ?, ?, ?, ? )''', ( start, email, sent_at, subject, hdr, body))
    if count % 50 == 0 : conn.commit()
    if count % 100 == 0 : time.sleep(1)

conn.commit()
cur.close()


How many messages:2
http://mbox.dr-chuck.net/sakai.devel/21/22 2625
    csev@umich.edu 2005-12-12T11:33:29-05:00 sepp library discussion group renamed - now with open membership
http://mbox.dr-chuck.net/sakai.devel/22/23 2960
    pgoldweic@northwestern.edu 2005-12-12T13:16:12-06:00 problems accessing the collab site with internet explorer...


KeyboardInterrupt: ignored

In [19]:
# gmodel.py

import sqlite3
import time
import ssl
import urllib.request, urllib.parse, urllib.error
from urllib.parse import urljoin
from urllib.parse import urlparse
import re
from datetime import datetime, timedelta

# Not all systems have this so conditionally define parser
try:
    import dateutil.parser as parser
except:
    pass

def parsemaildate(md) :
    # See if we have dateutil
    try:
        pdate = parser.parse(tdate)
        test_at = pdate.isoformat()
        return test_at
    except:
        pass

    # Non-dateutil version - we try our best

    pieces = md.split()
    notz = " ".join(pieces[:4]).strip()

    # Try a bunch of format variations - strptime() is *lame*
    dnotz = None
    for form in [ '%d %b %Y %H:%M:%S', '%d %b %Y %H:%M:%S',
        '%d %b %Y %H:%M', '%d %b %Y %H:%M', '%d %b %y %H:%M:%S',
        '%d %b %y %H:%M:%S', '%d %b %y %H:%M', '%d %b %y %H:%M' ] :
        try:
            dnotz = datetime.strptime(notz, form)
            break
        except:
            continue

    if dnotz is None :
        # print 'Bad Date:',md
        return None

    iso = dnotz.isoformat()

    tz = "+0000"
    try:
        tz = pieces[4]
        ival = int(tz) # Only want numeric timezone values
        if tz == '-0000' : tz = '+0000'
        tzh = tz[:3]
        tzm = tz[3:]
        tz = tzh+":"+tzm
    except:
        pass

    return iso+tz

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

conn = sqlite3.connect('content.sqlite')
cur = conn.cursor()

baseurl = "http://mbox.dr-chuck.net/sakai.devel/"

cur.execute('''CREATE TABLE IF NOT EXISTS Messages
    (id INTEGER UNIQUE, email TEXT, sent_at TEXT,
     subject TEXT, headers TEXT, body TEXT)''')

# Pick up where we left off
start = None
cur.execute('SELECT max(id) FROM Messages' )
try:
    row = cur.fetchone()
    if row is None :
        start = 0
    else:
        start = row[0]
except:
    start = 0

if start is None : start = 0

many = 0
count = 0
fail = 0
while True:
    if ( many < 1 ) :
        conn.commit()
        sval = input('How many messages:')
        if ( len(sval) < 1 ) : break
        many = int(sval)

    start = start + 1
    cur.execute('SELECT id FROM Messages WHERE id=?', (start,) )
    try:
        row = cur.fetchone()
        if row is not None : continue
    except:
        row = None

    many = many - 1
    url = baseurl + str(start) + '/' + str(start + 1)

    text = "None"
    try:
        # Open with a timeout of 30 seconds
        document = urllib.request.urlopen(url, None, 30, context=ctx)
        text = document.read().decode()
        if document.getcode() != 200 :
            print("Error code=",document.getcode(), url)
            break
    except KeyboardInterrupt:
        print('')
        print('Program interrupted by user...')
        break
    except Exception as e:
        print("Unable to retrieve or parse page",url)
        print("Error",e)
        fail = fail + 1
        if fail > 5 : break
        continue

    print(url,len(text))
    count = count + 1

    if not text.startswith("From "):
        print(text)
        print("Did not find From ")
        fail = fail + 1
        if fail > 5 : break
        continue

    pos = text.find("\n\n")
    if pos > 0 :
        hdr = text[:pos]
        body = text[pos+2:]
    else:
        print(text)
        print("Could not find break between headers and body")
        fail = fail + 1
        if fail > 5 : break
        continue

    email = None
    x = re.findall('\nFrom: .* <(\S+@\S+)>\n', hdr)
    if len(x) == 1 :
        email = x[0];
        email = email.strip().lower()
        email = email.replace("<","")
    else:
        x = re.findall('\nFrom: (\S+@\S+)\n', hdr)
        if len(x) == 1 :
            email = x[0];
            email = email.strip().lower()
            email = email.replace("<","")

    date = None
    y = re.findall('\Date: .*, (.*)\n', hdr)
    if len(y) == 1 :
        tdate = y[0]
        tdate = tdate[:26]
        try:
            sent_at = parsemaildate(tdate)
        except:
            print(text)
            print("Parse fail",tdate)
            fail = fail + 1
            if fail > 5 : break
            continue

    subject = None
    z = re.findall('\Subject: (.*)\n', hdr)
    if len(z) == 1 : subject = z[0].strip().lower();

    # Reset the fail counter
    fail = 0
    print("   ",email,sent_at,subject)
    cur.execute('''INSERT OR IGNORE INTO Messages (id, email, sent_at, subject, headers, body)
        VALUES ( ?, ?, ?, ?, ?, ? )''', ( start, email, sent_at, subject, hdr, body))
    if count % 50 == 0 : conn.commit()
    if count % 100 == 0 : time.sleep(1)

conn.commit()
cur.close()


http://mbox.dr-chuck.net/sakai.devel/23/24 4715
    pgoldweic@northwestern.edu 2005-12-12T14:20:41-06:00 re: problems accessing the collab site with internet
http://mbox.dr-chuck.net/sakai.devel/24/25 2705
    andrew@caret.cam.ac.uk 2005-12-12T14:23:12+00:00 file picker for non-legacy tools


KeyboardInterrupt: ignored